In [1]:
from project.KEM import *
from project.hash import *
from project.polynomial import *

# Testy funkcji potrzebnych do generacji kluczy

In [2]:
d_seeds = get_seeds(SeedPurpose.KEYGEN_SEEDS)

In [3]:
print(d_seeds.s1.raw)
print(d_seeds.s2.raw)

[196 177  64 240 194 198  92 100  18 149  41  63 204 197 114 223  40 191
  17 229 220 205 228 215  70 200 167   4  95 195  39 177]
[196 177  64 240 194 198  92 100  18 149  41  63 204 197 114 223  40 191
  17 229 220 205 228 215  70 200 167   4  95 195  39 177]


In [4]:
state = shake256_init(d_seeds.s1.raw, ELLSIZE)

In [5]:
print(state)
print(len(state.buffer))

SHAstate(buffer=bytearray(b'\x82\x03\xbe\x95\xb0\x9fw\xc5|2;3\x0b\x0f\x93\xb9/W\xd2r\x1d\xdb\xf9\x89\x88\x0c\xb6\xd1\xea\xc4B\x18q@\x06r\x98\r\xcf*\xb7lT\x00\xfd\xd4\x0c;IM?\xb8\x15\x0c\xf6O+>\xda>\x1a{t\xd0\xe2\xa5\x06r\x81\xcbjC\x9b\x04\x9e\xe0\x9c\x97\xad\xcbLO\xb3\xf5\x92_\xa4\xf9\xcc\rh\xee\x17bNV\xf8"\x84k\x8fLvV1K\x89\xbd\xbb\xa7y\xfc\xbfJ\x04\x05\xb0\x03\x89z$\xef\xd1\xf9\x04\xf7\xe6\x10n\xb6\xa1\xcc\xb3\xb9\xd3;\x0e\xfd1@\x04\x13i_\x1f=\xb9\xabE\x85\xe5P\x1d\xb8\xdb9Z\xb5_\xb4\x03\xa6\xe6\x19\xc2\xff\xae\xf1Zy\x82:-I\x833\x8d\xa3\xfd\xd2M\x8c\x8bBO\x02\xdcXJ\xaa\xa0\xae\xb1|\xce\'\x81\x9a\x07H'), pos=0)
200


In [6]:
h0 = generate_sparse_rep_keccak(DV, RBITS, state)

In [7]:
h1 = generate_sparse_rep_keccak(DV, RBITS, state)

In [8]:
print(h0)

[254 255 255 ...   0   0   0]


In [9]:
print(h1)

[255 255 255 ...   0   0   0]


# Test generacji kluczy

In [10]:
pk, sk = generate_keys()

In [11]:
print(pk.value)
print(len(pk.value))

[ 49 144 230 ... 222 160 146]
5122


In [12]:
print(sk.val0)
print(len(sk.val0))

[254 255 255 ...   0   0   0]
5122


In [13]:
print(sk.val1)
print(len(sk.val1))

[255 255 255 ...   0   0   0]
5122


# Testy funkcji potrzebnych do enkapsulacji

In [14]:
d_seeds = get_seeds(SeedPurpose.KEYGEN_SEEDS)

In [15]:
m = d_seeds.s1.raw
e = function_h(m)

In [16]:
print(len(e))

10244


In [17]:
l = len(e)-RSIZE

In [18]:
e0, e1 = split_polynomial(e)

In [19]:
print(e0)
print(len(e0))
print(e1)
print(len(e1))

[255 255 255 ...   0   0   0]
5122
[0 0 0 ... 0 0 0]
5122


In [20]:
c0 = poly_add(e0, mod_mul(e1, pk.value))

In [21]:
print(c0)

[250  46 180 ...  73 255  56]


# Test działania encapsulacji

In [22]:
ct, ss = encapsulate(pk)

In [23]:
print(ct.val0)
print(ct.val1)

[177 233  68 ... 120  90 169]
[228  21 132  51 115 247   6 236  79 178 174 118 189 209 208 193 138  28
  25  52  12   0 117 140 243 113  14  52 119 183 189  53]


In [24]:
print(ss.raw)

[ 43 172  22 242 211 173 228  36 212 252  38 174  33 159 185 162  24 188
 157  12 199 119  34 255 153 184  25 233 219 103 186 185]


# Testy funkcji potrzebnych do dekapsulacji

In [25]:
cm0 = convert2compact(sk.val0)

In [26]:
cm1 = convert2compact(sk.val1)

In [27]:
syndrome = compute_syndrome(ct, sk)

In [28]:
print(syndrome)

Syndrome(raw=array([1, 0, 0, ..., 1, 1, 1], dtype=uint8))


In [29]:
e_t = BGF_decoder(syndrome, cm0, cm1)

In [30]:
print(e_t)

(1, array([0, 0, 0, ..., 0, 0, 0], dtype=uint8))


In [31]:
e_p = convertbin2byte(e_t[1], 2 * RBITS)

In [32]:
print(e_p)

[0 0 0 ... 0 0 0]


In [33]:
e_l = function_l(e_p)

In [34]:
print(e_l)

[185 167  67  76 194 149 115 232 182 114  26 157 211  46 230 126 162 148
 155  87  72 184  78 178 157 253  76  78 185 101  57   8]


In [35]:
m_prime = np.zeros(ELLSIZE, dtype=np.uint8)
for i in range(ELLSIZE):
    m_prime[i] = ct.val1[i] ^ e_l[i]

In [36]:
e_e = function_h(m_prime)

In [37]:
print(e_e)

[255 255 255 ...   0   0   0]


In [38]:
eq = True
for i in range(len(e_e)):
    if e_e[i] != e_p[i]:
        eq = False

In [39]:
print(eq)

False


In [40]:
k1 = function_k(m_prime, ct.val0, ct.val1)

In [41]:
k2 = function_k(sk.sigma, ct.val0, ct.val1)

In [42]:
print(k1)

[ 49  75   8 169 108  18  39  72  53 252 152 233  11 163  16 228  43 100
 102 243 114  21 201   5  59 142 137 154  16 111  12 248]


In [43]:
print(k2)

[ 43 172  22 242 211 173 228  36 212 252  38 174  33 159 185 162  24 188
 157  12 199 119  34 255 153 184  25 233 219 103 186 185]


# Test działania całości - generowanie kluczy, enkapsulacja, dekapsulacja

In [44]:
pk, sk = generate_keys()

In [45]:
ct, ss = encapsulate(pk)

In [46]:
print(ss.raw)

[ 63 117  22 228  64 172  37  61  79 175  50  37  26  87  87 143  63  38
   2 121 125  52 239  38 123  62 157 228 106 107 171  41]


In [47]:
ss_r = decapsulate(ct, sk)

In [48]:
print(ss_r.raw)

[ 63 117  22 228  64 172  37  61  79 175  50  37  26  87  87 143  63  38
   2 121 125  52 239  38 123  62 157 228 106 107 171  41]
